In [1]:
pip install numpy pandas scikit-learn tensorflow matplotlib seaborn pyquaternion


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Adım 1: Gerekli Kütüphaneleri Yükleme
import os
import numpy as np
import pickle
from pyquaternion import Quaternion # Normalizasyon için
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# --- KONFİGÜRASYON ---
# Verinin bulunduğu ana klasör
DATA_FOLDER = '' 
# Hangi hareketi eğitmek istiyoruz?
MOVEMENT_TYPE = ''
# Tüm zaman serilerini eşitleyeceğimiz uzunluk (gördüğümüz 80-82'ye göre 80 ideal)
TIMESTEPS = 80
NUM_SENSORS = 10
FEATURES_PER_SENSOR = 4 # w, x, y, z
FEATURES = NUM_SENSORS * FEATURES_PER_SENSOR # Toplam 40 özellik
USERS_TO_ANALYZE = ["User-A","User-B", "User-E"]


# --- Adım 2.1: Normalizasyon için Yardımcı Fonksiyon ---

def normalize_sequence(sequence_data):
    """
    (N, 40) şeklindeki bir NumPy dizisini ilk karesine göre normalize eder.
    Sıralamanın (w, x, y, z) olduğunu varsayar.
    """
    if not sequence_data.any(): # Tamamen sıfırsa dokunma
        return sequence_data
    
    # 1. İlk kareyi (referans yönelimi) al
    first_frame = sequence_data[0] # (40,) şeklinde
    inverse_references = []
    
    # 2. 10 sensör için de ters (inverse) quaternion'ları hesapla
    for i in range(NUM_SENSORS):
        offset = i * FEATURES_PER_SENSOR
        w, x, y, z = first_frame[offset], first_frame[offset+1], first_frame[offset+2], first_frame[offset+3]
        
        # Olası bir sıfır veriye karşı koruma
        if w == 0 and x == 0 and y == 0 and z == 0:
             q_ref = Quaternion(1, 0, 0, 0) # Nötr (identity) quaternion
        else:
            q_ref = Quaternion(w, x, y, z)
            
        inverse_references.append(q_ref.inverse)
        
    normalized_sequence = []
    
    # 3. Tüm zaman adımlarını (timesteps) bu referanslara göre normalize et
    for frame in sequence_data:
        normalized_frame_features = []
        for i in range(NUM_SENSORS):
            offset = i * FEATURES_PER_SENSOR
            w, x, y, z = frame[offset], frame[offset+1], frame[offset+2], frame[offset+3]
            
            if w == 0 and x == 0 and y == 0 and z == 0:
                q_live = Quaternion(1, 0, 0, 0)
            else:
                q_live = Quaternion(w, x, y, z)
            
            # Referans yönelimi çıkar
            q_normalized = inverse_references[i] * q_live
            
            # Yeni (w,x,y,z) değerlerini düz listeye ekle
            normalized_frame_features.extend([q_normalized.w, q_normalized.x, q_normalized.y, q_normalized.z])
        
        normalized_sequence.append(normalized_frame_features)
        
    return np.array(normalized_sequence)


# --- Adım 2: Veriyi Yükleme ve Ön İşleme ---
print("Adım 2: Veri Yükleme ve Ön İşleme Başladı...")

all_data = []
all_labels = []

movement_path = os.path.join(DATA_FOLDER, MOVEMENT_TYPE)
# User-A, User-B, ... klasörlerinde gezin
for user_folder in USERS_TO_ANALYZE:
    user_path = os.path.join(movement_path, user_folder)
    if os.path.isdir(user_path) and user_folder.startswith('User-'):

        # True, WrongElbow, ... klasörlerinde gezin
        for label_folder in os.listdir(user_path):
            label_path = os.path.join(user_path, label_folder)
            if os.path.isdir(label_path):

                # .pkl dosyalarını oku
                for pkl_file in os.listdir(label_path):
                    if pkl_file.endswith('.pkl'):
                        file_path = os.path.join(label_path, pkl_file)

                        try:
                            with open(file_path, 'rb') as f:
                                obj = pickle.load(f)

                            # .pkl içindeki 'data' anahtarından veriyi al
                            recording_data = np.array(obj['data'])

                            # Veri geçerli mi diye kontrol et (en az 1 frame olmalı)
                            if recording_data.shape[0] < 1 or recording_data.shape[1] != FEATURES:
                                print(f"Uyarı: Hatalı veri atlanıyor (şekil {recording_data.shape}): {file_path}")
                                continue

                            # --- YENİ: Normalizasyon ---
                            recording_data_normalized = normalize_sequence(recording_data)

                            all_data.append(recording_data_normalized)
                            all_labels.append(label_folder) # Etiket olarak klasör adını kullan

                        except Exception as e:
                            print(f"Hata: {file_path} dosyası okunurken hata oluştu: {e}")

print(f"Toplam {len(all_data)} adet .pkl kaydı yüklendi.")

# --- Adım 3: Boyut Eşitleme (Padding/Trimming) ---
print(f"Adım 3: Tüm veriler {TIMESTEPS} adımına eşitleniyor...")

# Keras'ın 'pad_sequences' fonksiyonunu kullanalım. Çok daha verimli.
# 'post' = eksik veriyi sona ekle/sondan kırp
X_padded = pad_sequences(
    all_data, 
    maxlen=TIMESTEPS, 
    dtype='float32', 
    padding='post', 
    truncating='post'
)

print("Boyut Eşitleme Tamamlandı.\n")


# --- Adım 4: Veriyi Derin Öğrenme Formatına Dönüştürme ---
print("Adım 4: Veriyi Derin Öğrenme Formatına Dönüştürme Başladı...")

X = X_padded # Artık tüm verilerimiz (sample_sayısı, 80, 40) şeklinde
y = np.array(all_labels)

# Etiketleri sayısal değerlere çevir (örn: 'Correct' -> 0, 'WrongElbow' -> 1)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Sınıf sayısını al
num_classes = len(label_encoder.classes_)
print(f"Bulunan sınıflar ({num_classes} adet): {label_encoder.classes_}")

# Sayısal etiketleri one-hot encoding formatına çevir
# Örn: 3 sınıf varsa, 1 -> [0, 1, 0]
y_one_hot = to_categorical(y_encoded, num_classes=num_classes)

print(f"Veri şekli (X): {X.shape}")
print(f"Etiket şekli (y): {y_one_hot.shape}")
print("Veri Dönüştürme Tamamlandı.\n")


# --- Adım 5: Eğitim ve Test Verisi Olarak Ayırma ve Ölçekleme ---
print("Adım 5: Veriyi Ayırma ve Ölçekleme Başladı...")

X_train, X_test, y_train, y_test = train_test_split(
    X, y_one_hot, 
    test_size=0.2,       # Verinin %20'sini test için ayır
    random_state=42,     # Tekrarlanabilir sonuçlar için
    stratify=y_one_hot   # Sınıf dağılımını koru (çok önemli)
)

# Ölçekleme (Scaling) için veriyi 2D'ye çevirmemiz gerekiyor
# (sample_sayısı, 80, 40) -> (sample_sayısı * 80, 40)
scaler = StandardScaler()
X_train_reshaped = X_train.reshape(-1, FEATURES)
scaler.fit(X_train_reshaped)

X_train_scaled_reshaped = scaler.transform(X_train_reshaped)
X_test_scaled_reshaped = scaler.transform(X_test.reshape(-1, FEATURES))

# Veriyi tekrar 3D (LSTM formatı) haline getirelim
X_train_scaled = X_train_scaled_reshaped.reshape(X_train.shape)
X_test_scaled = X_test_scaled_reshaped.reshape(X_test.shape)

print(f"Eğitim verisi: {X_train_scaled.shape}, Test verisi: {X_test_scaled.shape}")
print("Veri Ayırma ve Ölçekleme Tamamlandı.\n")


# --- Adım 6: LSTM Modelini Oluşturma ---
print("Adım 6: LSTM Modeli Oluşturuluyor...")

model = Sequential([
    # Input katmanı: (80, 40) şeklinde veri alacak
    LSTM(64, input_shape=(TIMESTEPS, FEATURES), return_sequences=True),
    BatchNormalization(), # Katmanlar arası normalizasyon, öğrenmeyi hızlandırır
    Dropout(0.3),
    
    LSTM(32, return_sequences=False), # Son LSTM katmanı, sadece son çıktıyı verir
    BatchNormalization(),
    Dropout(0.3),
    
    Dense(32, activation='relu'),
    
    # Çıkış katmanı: Sınıf sayısı kadar nöron ve 'softmax' aktivasyonu
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
print("Model Oluşturma Tamamlandı.\n")


# --- Adım 7: Modeli Eğitme ---
print("Adım 7: Model Eğitimi Başlıyor...")

history = model.fit(
    X_train_scaled, 
    y_train, 
    epochs=60, # Epoch sayısını 50-100 arası deneyebilirsin
    batch_size=16, 
    validation_data=(X_test_scaled, y_test)
)

print("Model Eğitimi Tamamlandı.\n")


# --- Adım 8: Modeli Değerlendirme ---
print("Adım 8: Model Değerlendirmesi...")

# Test verisi üzerindeki loss ve accuracy değerleri
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Tahminleri yap
y_pred_probs = model.predict(X_test_scaled)
y_pred = np.argmax(y_pred_probs, axis=1) # Olasılıklardan en yüksek sınıfı seç
y_test_labels = np.argmax(y_test, axis=1) # One-hot'tan normal etikete dön

# Sınıflandırma Raporu (Precision, Recall, F1-score)
print("\nClassification Report:")
print(classification_report(y_test_labels, y_pred, target_names=label_encoder.classes_))

# Karışıklık Matrisi (Confusion Matrix)
print("\nConfusion Matrix:")
cm = confusion_matrix(y_test_labels, y_pred)
plt.figure(figsize=(10, 7))
sns.heatmap(
    cm, annot=True, fmt='d', cmap='Blues', 
    xticklabels=label_encoder.classes_, 
    yticklabels=label_encoder.classes_
)
plt.xlabel('Tahmin Edilen (Predicted)')
plt.ylabel('Gerçek (True)')
plt.show()


# --- Adım 9: Modeli ve Gerekli Nesneleri Kaydetme ---
print("\nAdım 9: Model ve diğer nesneler kaydediliyor...")

MODEL_OUTPUT_DIR = f"final_lstm_models_{MOVEMENT_TYPE}"
if not os.path.exists(MODEL_OUTPUT_DIR):
    os.makedirs(MODEL_OUTPUT_DIR)

# 1. Keras modelini kaydet
model.save(os.path.join(MODEL_OUTPUT_DIR, f'{MOVEMENT_TYPE}_lstm_model.h5'))

# 2. Scaler'ı kaydet
with open(os.path.join(MODEL_OUTPUT_DIR, f'{MOVEMENT_TYPE}_scaler.pkl'), 'wb') as f:
    pickle.dump(scaler, f)

# 3. Label Encoder'ı kaydet
with open(os.path.join(MODEL_OUTPUT_DIR, f'{MOVEMENT_TYPE}_label_encoder.pkl'), 'wb') as f:
    pickle.dump(label_encoder, f)

print(f"Tüm dosyalar başarıyla '{MODEL_OUTPUT_DIR}' klasörüne kaydedildi.")

2025-10-27 20:03:04.789114: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-27 20:03:04.789611: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-27 20:03:04.869764: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-27 20:03:06.344087: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tur

Adım 2: Veri Yükleme ve Ön İşleme Başladı...
Toplam 131 adet .pkl kaydı yüklendi.
Adım 3: Tüm veriler 80 adımına eşitleniyor...
Boyut Eşitleme Tamamlandı.

Adım 4: Veriyi Derin Öğrenme Formatına Dönüştürme Başladı...
Bulunan sınıflar (3 adet): ['RoundedBack' 'SupportHand' 'True']
Veri şekli (X): (131, 80, 40)
Etiket şekli (y): (131, 3)
Veri Dönüştürme Tamamlandı.

Adım 5: Veriyi Ayırma ve Ölçekleme Başladı...
Eğitim verisi: (104, 80, 40), Test verisi: (27, 80, 40)
Veri Ayırma ve Ölçekleme Tamamlandı.

Adım 6: LSTM Modeli Oluşturuluyor...


2025-10-27 20:03:08.657300: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/opt/conda/envs/python313/lib/python3.13/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 80, 64)         │        26,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 80, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 80, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 40,835 (159.51 KB)

 Trainable params: 40,643 (158.76 KB)

 Non-trainable params: 192 (768.00 B)

Model Oluşturma Tamamlandı.

Adım 7: Model Eğitimi Başlıyor...
Epoch 1/60
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.3416 - loss: 1.8520

In [ ]:
# Adım 1: Gerekli Kütüphaneleri Yükleme
import os
import numpy as np
import pickle
from pyquaternion import Quaternion # Normalizasyon için
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from tensorflow.keras.layers import GRU # <-- Değiştir
import matplotlib.pyplot as plt

# --- KONFİGÜRASYON ---
# Verinin bulunduğu ana klasör
DATA_FOLDER = '' 
# Hangi hareketi eğitmek istiyoruz?
MOVEMENT_TYPE = ''
# Tüm zaman serilerini eşitleyeceğimiz uzunluk (gördüğümüz 80-82'ye göre 80 ideal)
TIMESTEPS = 80
NUM_SENSORS = 10
FEATURES_PER_SENSOR = 4 # w, x, y, z
FEATURES = NUM_SENSORS * FEATURES_PER_SENSOR # Toplam 40 özellik
USERS_TO_ANALYZE = ["User-A","User-B", "User-E"]


# --- Adım 2.1: Normalizasyon için Yardımcı Fonksiyon ---

def normalize_sequence(sequence_data):
    """
    (N, 40) şeklindeki bir NumPy dizisini ilk karesine göre normalize eder.
    Sıralamanın (w, x, y, z) olduğunu varsayar.
    """
    if not sequence_data.any(): # Tamamen sıfırsa dokunma
        return sequence_data
    
    # 1. İlk kareyi (referans yönelimi) al
    first_frame = sequence_data[0] # (40,) şeklinde
    inverse_references = []
    
    # 2. 10 sensör için de ters (inverse) quaternion'ları hesapla
    for i in range(NUM_SENSORS):
        offset = i * FEATURES_PER_SENSOR
        w, x, y, z = first_frame[offset], first_frame[offset+1], first_frame[offset+2], first_frame[offset+3]
        
        # Olası bir sıfır veriye karşı koruma
        if w == 0 and x == 0 and y == 0 and z == 0:
             q_ref = Quaternion(1, 0, 0, 0) # Nötr (identity) quaternion
        else:
            q_ref = Quaternion(w, x, y, z)
            
        inverse_references.append(q_ref.inverse)
        
    normalized_sequence = []
    
    # 3. Tüm zaman adımlarını (timesteps) bu referanslara göre normalize et
    for frame in sequence_data:
        normalized_frame_features = []
        for i in range(NUM_SENSORS):
            offset = i * FEATURES_PER_SENSOR
            w, x, y, z = frame[offset], frame[offset+1], frame[offset+2], frame[offset+3]
            
            if w == 0 and x == 0 and y == 0 and z == 0:
                q_live = Quaternion(1, 0, 0, 0)
            else:
                q_live = Quaternion(w, x, y, z)
            
            # Referans yönelimi çıkar
            q_normalized = inverse_references[i] * q_live
            
            # Yeni (w,x,y,z) değerlerini düz listeye ekle
            normalized_frame_features.extend([q_normalized.w, q_normalized.x, q_normalized.y, q_normalized.z])
        
        normalized_sequence.append(normalized_frame_features)
        
    return np.array(normalized_sequence)


# --- Adım 2: Veriyi Yükleme ve Ön İşleme ---
print("Adım 2: Veri Yükleme ve Ön İşleme Başladı...")

all_data = []
all_labels = []

movement_path = os.path.join(DATA_FOLDER, MOVEMENT_TYPE)
# User-A, User-B, ... klasörlerinde gezin
for user_folder in USERS_TO_ANALYZE:
    user_path = os.path.join(movement_path, user_folder)
    if os.path.isdir(user_path) and user_folder.startswith('User-'):

        # True, WrongElbow, ... klasörlerinde gezin
        for label_folder in os.listdir(user_path):
            label_path = os.path.join(user_path, label_folder)
            if os.path.isdir(label_path):

                # .pkl dosyalarını oku
                for pkl_file in os.listdir(label_path):
                    if pkl_file.endswith('.pkl'):
                        file_path = os.path.join(label_path, pkl_file)

                        try:
                            with open(file_path, 'rb') as f:
                                obj = pickle.load(f)

                            # .pkl içindeki 'data' anahtarından veriyi al
                            recording_data = np.array(obj['data'])

                            # Veri geçerli mi diye kontrol et (en az 1 frame olmalı)
                            if recording_data.shape[0] < 1 or recording_data.shape[1] != FEATURES:
                                print(f"Uyarı: Hatalı veri atlanıyor (şekil {recording_data.shape}): {file_path}")
                                continue

                            # --- YENİ: Normalizasyon ---
                            recording_data_normalized = normalize_sequence(recording_data)

                            all_data.append(recording_data_normalized)
                            all_labels.append(label_folder) # Etiket olarak klasör adını kullan

                        except Exception as e:
                            print(f"Hata: {file_path} dosyası okunurken hata oluştu: {e}")

print(f"Toplam {len(all_data)} adet .pkl kaydı yüklendi.")

# --- Adım 3: Boyut Eşitleme (Padding/Trimming) ---
print(f"Adım 3: Tüm veriler {TIMESTEPS} adımına eşitleniyor...")

# Keras'ın 'pad_sequences' fonksiyonunu kullanalım. Çok daha verimli.
# 'post' = eksik veriyi sona ekle/sondan kırp
X_padded = pad_sequences(
    all_data, 
    maxlen=TIMESTEPS, 
    dtype='float32', 
    padding='post', 
    truncating='post'
)

print("Boyut Eşitleme Tamamlandı.\n")


# --- Adım 4: Veriyi Derin Öğrenme Formatına Dönüştürme ---
print("Adım 4: Veriyi Derin Öğrenme Formatına Dönüştürme Başladı...")

X = X_padded # Artık tüm verilerimiz (sample_sayısı, 80, 40) şeklinde
y = np.array(all_labels)

# Etiketleri sayısal değerlere çevir (örn: 'Correct' -> 0, 'WrongElbow' -> 1)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Sınıf sayısını al
num_classes = len(label_encoder.classes_)
print(f"Bulunan sınıflar ({num_classes} adet): {label_encoder.classes_}")

# Sayısal etiketleri one-hot encoding formatına çevir
# Örn: 3 sınıf varsa, 1 -> [0, 1, 0]
y_one_hot = to_categorical(y_encoded, num_classes=num_classes)

print(f"Veri şekli (X): {X.shape}")
print(f"Etiket şekli (y): {y_one_hot.shape}")
print("Veri Dönüştürme Tamamlandı.\n")


# --- Adım 5: Eğitim ve Test Verisi Olarak Ayırma ve Ölçekleme ---
print("Adım 5: Veriyi Ayırma ve Ölçekleme Başladı...")

X_train, X_test, y_train, y_test = train_test_split(
    X, y_one_hot, 
    test_size=0.2,       # Verinin %20'sini test için ayır
    random_state=42,     # Tekrarlanabilir sonuçlar için
    stratify=y_one_hot   # Sınıf dağılımını koru (çok önemli)
)

# Ölçekleme (Scaling) için veriyi 2D'ye çevirmemiz gerekiyor
# (sample_sayısı, 80, 40) -> (sample_sayısı * 80, 40)
scaler = StandardScaler()
X_train_reshaped = X_train.reshape(-1, FEATURES)
scaler.fit(X_train_reshaped)

X_train_scaled_reshaped = scaler.transform(X_train_reshaped)
X_test_scaled_reshaped = scaler.transform(X_test.reshape(-1, FEATURES))

# Veriyi tekrar 3D (LSTM formatı) haline getirelim
X_train_scaled = X_train_scaled_reshaped.reshape(X_train.shape)
X_test_scaled = X_test_scaled_reshaped.reshape(X_test.shape)

print(f"Eğitim verisi: {X_train_scaled.shape}, Test verisi: {X_test_scaled.shape}")
print("Veri Ayırma ve Ölçekleme Tamamlandı.\n")


# --- Adım 6: LSTM Modelini Oluşturma ---
print("Adım 6: LSTM Modeli Oluşturuluyor...")

model = Sequential([
    # Input katmanı: (80, 40) şeklinde veri alacak
    GRU(64, input_shape=(TIMESTEPS, FEATURES), return_sequences=True),
    BatchNormalization(), # Katmanlar arası normalizasyon, öğrenmeyi hızlandırır
    Dropout(0.3),
    
    GRU(32, return_sequences=False), # Son LSTM katmanı, sadece son çıktıyı verir
    BatchNormalization(),
    Dropout(0.3),
    
    Dense(32, activation='relu'),
    
    # Çıkış katmanı: Sınıf sayısı kadar nöron ve 'softmax' aktivasyonu
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
print("Model Oluşturma Tamamlandı.\n")


# --- Adım 7: Modeli Eğitme ---
print("Adım 7: Model Eğitimi Başlıyor...")

history = model.fit(
    X_train_scaled, 
    y_train, 
    epochs=60, # Epoch sayısını 50-100 arası deneyebilirsin
    batch_size=16, 
    validation_data=(X_test_scaled, y_test)
)

print("Model Eğitimi Tamamlandı.\n")


# --- Adım 8: Modeli Değerlendirme ---
print("Adım 8: Model Değerlendirmesi...")

# Test verisi üzerindeki loss ve accuracy değerleri
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Tahminleri yap
y_pred_probs = model.predict(X_test_scaled)
y_pred = np.argmax(y_pred_probs, axis=1) # Olasılıklardan en yüksek sınıfı seç
y_test_labels = np.argmax(y_test, axis=1) # One-hot'tan normal etikete dön

# Sınıflandırma Raporu (Precision, Recall, F1-score)
print("\nClassification Report:")
print(classification_report(y_test_labels, y_pred, target_names=label_encoder.classes_))

# Karışıklık Matrisi (Confusion Matrix)
print("\nConfusion Matrix:")
cm = confusion_matrix(y_test_labels, y_pred)
plt.figure(figsize=(10, 7))
sns.heatmap(
    cm, annot=True, fmt='d', cmap='Blues', 
    xticklabels=label_encoder.classes_, 
    yticklabels=label_encoder.classes_
)
plt.xlabel('Tahmin Edilen (Predicted)')
plt.ylabel('Gerçek (True)')
plt.show()


# --- Adım 9: Modeli ve Gerekli Nesneleri Kaydetme ---
print("\nAdım 9: Model ve diğer nesneler kaydediliyor...")

MODEL_OUTPUT_DIR = f"final_lstm_models_{MOVEMENT_TYPE}"
if not os.path.exists(MODEL_OUTPUT_DIR):
    os.makedirs(MODEL_OUTPUT_DIR)

# 1. Keras modelini kaydet
model.save(os.path.join(MODEL_OUTPUT_DIR, f'{MOVEMENT_TYPE}_lstm_model.h5'))

# 2. Scaler'ı kaydet
with open(os.path.join(MODEL_OUTPUT_DIR, f'{MOVEMENT_TYPE}_scaler.pkl'), 'wb') as f:
    pickle.dump(scaler, f)

# 3. Label Encoder'ı kaydet
with open(os.path.join(MODEL_OUTPUT_DIR, f'{MOVEMENT_TYPE}_label_encoder.pkl'), 'wb') as f:
    pickle.dump(label_encoder, f)

print(f"Tüm dosyalar başarıyla '{MODEL_OUTPUT_DIR}' klasörüne kaydedildi.")

In [7]:
# Adım 1: Gerekli Kütüphaneleri Yükleme
import os
import numpy as np
import pickle
import copy # Scaler'ı kopyalamak için
from pyquaternion import Quaternion # Normalizasyon için
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# TensorFlow ve Keras kütüphanelerini import etme
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    LSTM, GRU, Conv1D, MaxPooling1D, GlobalAveragePooling1D,
    Dense, Dropout, BatchNormalization, Input, LayerNormalization, 
    MultiHeadAttention, Add
)
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Flatten # <-- Ekle


# GPU hatalarını ve gereksiz uyarıları gizle
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
tf.get_logger().setLevel('ERROR')

# --- KONFİGÜRASYON ---
DATA_FOLDER = '' 
MOVEMENT_TYPE = '' # Test etmek istediğin hareket
TIMESTEPS = 80 # Veri setindeki (80, 40) şekline göre
NUM_SENSORS = 10
FEATURES_PER_SENSOR = 4 # w, x, y, z
FEATURES = NUM_SENSORS * FEATURES_PER_SENSOR
USERS_TO_ANALYZE = ["User-A", "User-B", "User-E"]

# Eğitim Ayarları
EPOCHS = 100 # Overfitting'i önlemek için EarlyStopping kullanacağız
BATCH_SIZE = 16

# --- Adım 2: Yardımcı Fonksiyonlar (Veri İşleme) ---

def normalize_sequence(sequence_data):
    """
    (N, 40) şeklindeki bir NumPy dizisini ilk karesine göre normalize eder.
    (w, x, y, z) sırasına göre.
    """
    if not sequence_data.any(): return sequence_data
    first_frame = sequence_data[0]
    inverse_references = []
    for i in range(NUM_SENSORS):
        offset = i * FEATURES_PER_SENSOR
        w, x, y, z = first_frame[offset:offset+4]
        q_ref = Quaternion(w, x, y, z) if (w or x or y or z) else Quaternion(1, 0, 0, 0)
        inverse_references.append(q_ref.inverse)
        
    normalized_sequence = []
    for frame in sequence_data:
        normalized_frame_features = []
        for i in range(NUM_SENSORS):
            offset = i * FEATURES_PER_SENSOR
            w, x, y, z = frame[offset:offset+4]
            q_live = Quaternion(w, x, y, z) if (w or x or y or z) else Quaternion(1, 0, 0, 0)
            q_normalized = inverse_references[i] * q_live
            normalized_frame_features.extend([q_normalized.w, q_normalized.x, q_normalized.y, q_normalized.z])
        normalized_sequence.append(normalized_frame_features)
    return np.array(normalized_sequence)

def augment_data(sequence_data, noise_level=0.01):
    """Veriye rastgele küçük gürültüler (jitter) ekler."""
    noise = np.random.normal(0, noise_level, sequence_data.shape)
    return sequence_data + noise

# --- Adım 3: Model Mimarilerini Oluşturan Fonksiyonlar ---

INPUT_SHAPE = (TIMESTEPS, FEATURES)

def build_cnn_lstm_model(num_classes):
    """Senin başarılı CNN-LSTM Hibrid modelin."""
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=INPUT_SHAPE),
        Conv1D(filters=64, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        BatchNormalization(),
        Dropout(0.3),
        LSTM(50, return_sequences=False),
        BatchNormalization(),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dense(num_classes, activation='softmax')
    ], name="CNN_LSTM")
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def build_cnn_ann_model(num_classes):
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=INPUT_SHAPE),
        MaxPooling1D(pool_size=2),
        BatchNormalization(),
        Dropout(0.3),
        
        Conv1D(filters=128, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        BatchNormalization(),
        Dropout(0.3),
        
        Flatten(), # <-- GlobalAveragePooling yerine Flatten
        
        Dense(100, activation='relu'),
        Dropout(0.4),
        Dense(num_classes, activation='softmax')
    ], name="CNN_ANN_Hybrid")
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def build_pure_cnn_model(num_classes):
    """Sadece 1D-CNN katmanları kullanan model."""
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=INPUT_SHAPE),
        BatchNormalization(),
        Conv1D(filters=64, kernel_size=3, activation='relu'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),
        
        Conv1D(filters=128, kernel_size=3, activation='relu'),
        BatchNormalization(),
        Conv1D(filters=128, kernel_size=3, activation='relu'),
        BatchNormalization(),
        GlobalAveragePooling1D(), # <-- LSTM/GRU yerine
        
        Dropout(0.4),
        Dense(64, activation='relu'),
        Dense(num_classes, activation='softmax')
    ], name="Pure_1D_CNN")
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def build_transformer_model(num_classes):
    """Basit bir Transformer Encoder bloğu kullanan model."""
    inputs = Input(shape=INPUT_SHAPE)
    
    # 1. Önce CNN ile özellikleri çıkaralım (Transformer'a yardımcı olur)
    x = Conv1D(filters=64, kernel_size=3, activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    
    # --- Transformer Encoder Bloğu ---
    # 2. Multi-Head Attention
    attn_output = MultiHeadAttention(num_heads=4, key_dim=64)(x, x)
    attn_output = Dropout(0.2)(attn_output)
    x = Add()([x, attn_output]) # Residual Connection (Toplama)
    x = LayerNormalization(epsilon=1e-6)(x)
    
    # 3. Feed Forward Kısmı
    ff_output = Dense(128, activation='relu')(x)
    ff_output = Dropout(0.2)(ff_output)
    ff_output = Dense(64, activation='relu')(ff_output)
    x = Add()([x, ff_output]) # Residual Connection
    x = LayerNormalization(epsilon=1e-6)(x)
    # --- Blok Sonu ---
    
    x = GlobalAveragePooling1D()(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs, name="Transformer")
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


# --- Adım 4: Veriyi Yükleme ve Ön İşleme ---
print("Adım 4: Veri Yükleme ve Ön İşleme Başladı...")
data_by_user = {}
all_labels_flat = []

movement_path = os.path.join(DATA_FOLDER, MOVEMENT_TYPE)
for user_folder in USERS_TO_ANALYZE:
    user_path = os.path.join(movement_path, user_folder)
    if not os.path.isdir(user_path):
        print(f"Uyarı: Kullanıcı klasörü atlanıyor (bulunamadı): {user_path}")
        continue
    
    print(f"Kullanıcı yükleniyor: {user_folder}")
    data_by_user[user_folder] = {'data': [], 'labels': []}
    
    for label_folder in os.listdir(user_path):
        label_path = os.path.join(user_path, label_folder)
        if os.path.isdir(label_path):
            for pkl_file in os.listdir(label_path):
                if pkl_file.endswith('.pkl'):
                    file_path = os.path.join(label_path, pkl_file)
                    try:
                        with open(file_path, 'rb') as f:
                            obj = pickle.load(f)
                        recording_data = np.array(obj['data'])
                        if recording_data.shape[0] < 1 or recording_data.shape[1] != FEATURES:
                            continue
                        
                        # --- Normalizasyon ve Boyut Eşitleme ---
                        recording_data_normalized = normalize_sequence(recording_data)
                        recording_data_padded = pad_sequences(
                            [recording_data_normalized], maxlen=TIMESTEPS, dtype='float32', 
                            padding='post', truncating='post'
                        )[0] # [0] ile (1, 80, 40) yerine (80, 40) al
                        
                        data_by_user[user_folder]['data'].append(recording_data_padded)
                        data_by_user[user_folder]['labels'].append(label_folder)
                        all_labels_flat.append(label_folder)
                    except Exception as e:
                        print(f"Hata: {file_path} dosyası okunurken hata oluştu: {e}")

print("Veri Yükleme Tamamlandı.\n")

# --- Adım 5: Etiket Kodlayıcıyı Hazırlama ---
label_encoder = LabelEncoder()
label_encoder.fit(all_labels_flat)
num_classes = len(label_encoder.classes_)
print(f"Bulunan sınıflar ({num_classes} adet): {label_encoder.classes_}\n")

# --- Adım 6: LOUOCV Test Çatısı ---
print("Adım 6: LOUOCV Test Çatısı Başlatılıyor...")

# Test edilecek modelleri ve oluşturma fonksiyonlarını bir sözlükte topla
models_to_test = {
    "CNN-LSTM": build_cnn_lstm_model,
    #"Pure 1D-CNN": build_pure_cnn_model,
    #"Transformer": build_transformer_model,
    #"ANN": build_cnn_ann_model
}

# Sonuçları saklamak için
overall_results = {}
all_classification_reports = {}

# Aşırı öğrenmeyi engellemek için Erken Durdurma
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Her bir model için döngü
for model_name, build_fn in models_to_test.items():
    print(f"\n==============================================")
    print(f"MODEL TEST EDİLİYOR: {model_name}")
    print(f"==============================================")
    
    fold_scores = []
    
    # LOUOCV döngüsü (her kullanıcı için)
    for user_to_test in USERS_TO_ANALYZE:
        print(f"\n--- Test Kullanıcısı: {user_to_test} ---")
        
        # 1. Eğitim ve Test Verisini Ayır
        X_train_list, y_train_list = [], []
        X_test_list, y_test_list = [], []

        for user, data in data_by_user.items():
            if user == user_to_test:
                X_test_list.extend(data['data'])
                y_test_list.extend(data['labels'])
            else:
                X_train_list.extend(data['data'])
                y_train_list.extend(data['labels'])

        # 2. Veri Artırma (Sadece Eğitim Verisine)
        X_train_aug, y_train_aug = [], []
        for i in range(len(X_train_list)):
            seq, lbl = X_train_list[i], y_train_list[i]
            X_train_aug.append(seq) # Orijinal
            y_train_aug.append(lbl)
            X_train_aug.append(augment_data(seq, 0.01)) # Gürültülü 1
            y_train_aug.append(lbl)
            X_train_aug.append(augment_data(seq, 0.005)) # Gürültülü 2
            y_train_aug.append(lbl)

        X_train = np.array(X_train_aug)
        X_test = np.array(X_test_list)
        
        # 3. Etiketleri Kodla
        y_train = to_categorical(label_encoder.transform(y_train_aug), num_classes=num_classes)
        y_test = to_categorical(label_encoder.transform(y_test_list), num_classes=num_classes)

        # 4. Ölçekle (Scaler'ı SADECE artırılmış eğitim verisine göre eğit)
        scaler = StandardScaler()
        X_train_reshaped = X_train.reshape(-1, FEATURES)
        scaler.fit(X_train_reshaped)
        
        X_train_scaled = scaler.transform(X_train_reshaped).reshape(X_train.shape)
        X_test_scaled = scaler.transform(X_test.reshape(-1, FEATURES)).reshape(X_test.shape)
        
        print(f"Eğitim verisi: {X_train_scaled.shape}, Test verisi: {X_test_scaled.shape}")

        # 5. Modeli Oluştur ve Eğit
        model = build_fn(num_classes)
        model.fit(
            X_train_scaled, y_train,
            epochs=EPOCHS,
            batch_size=BATCH_SIZE,
            validation_data=(X_test_scaled, y_test),
            callbacks=[early_stopping], # Erken durdurmayı kullan
            verbose=0 # Logları gizle
        )
        
        # 6. Değerlendir
        loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)
        fold_scores.append(accuracy)
        print(f"{user_to_test} için Doğruluk: {accuracy * 100:.2f}%")

    # Modelin ortalama LOUOCV skorunu kaydet
    overall_results[model_name] = np.mean(fold_scores)


# --- Adım 7: Nihai Sonuçları Göster ---
print("\n\n==============================================")
print("     LOUOCV GENEL SONUÇLARI     ")
print("==============================================")
print(f"Test Edilen Hareket: {MOVEMENT_TYPE}")
print(f"Veri Seti: {len(USERS_TO_ANALYZE)} Kullanıcı, Toplam {len(all_labels_flat)} Orijinal Kayıt")
print("----------------------------------------------")

for model_name, avg_accuracy in overall_results.items():
    print(f"Model: {model_name.ljust(12)} | Ortalama Genelleme Doğruluğu: {avg_accuracy * 100:.2f}%")

print("==============================================")

Adım 4: Veri Yükleme ve Ön İşleme Başladı...
Kullanıcı yükleniyor: User-A
Kullanıcı yükleniyor: User-B
Kullanıcı yükleniyor: User-E
Veri Yükleme Tamamlandı.

Bulunan sınıflar (3 adet): ['RoundedBack' 'SupportHand' 'True']

Adım 6: LOUOCV Test Çatısı Başlatılıyor...

MODEL TEST EDİLİYOR: CNN-LSTM

--- Test Kullanıcısı: User-A ---
Eğitim verisi: (258, 80, 40), Test verisi: (45, 80, 40)


/opt/conda/envs/python313/lib/python3.13/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


User-A için Doğruluk: 51.11%

--- Test Kullanıcısı: User-B ---
Eğitim verisi: (258, 80, 40), Test verisi: (45, 80, 40)


/opt/conda/envs/python313/lib/python3.13/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


User-B için Doğruluk: 84.44%

--- Test Kullanıcısı: User-E ---
Eğitim verisi: (270, 80, 40), Test verisi: (41, 80, 40)


/opt/conda/envs/python313/lib/python3.13/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


User-E için Doğruluk: 60.98%


     LOUOCV GENEL SONUÇLARI     
Test Edilen Hareket: 
Veri Seti: 3 Kullanıcı, Toplam 131 Orijinal Kayıt
----------------------------------------------
Model: CNN-LSTM     | Ortalama Genelleme Doğruluğu: 65.51%


In [16]:
# Adım 1: Gerekli Kütüphaneleri Yükleme
import os
import numpy as np
import pickle
from pyquaternion import Quaternion
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    LSTM, GRU, Conv1D, MaxPooling1D, GlobalAveragePooling1D,
    Dense, Dropout, BatchNormalization, Input, LayerNormalization, 
    MultiHeadAttention, Add
)
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.callbacks import EarlyStopping
import warnings
from tensorflow.keras.layers import Flatten # <-- Ekle
# Gereksiz TensorFlow ve Keras uyarılarını gizle
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
tf.get_logger().setLevel('ERROR')
warnings.filterwarnings('ignore', category=UserWarning) # Keras uyarılarını gizle

# --- KONFİGÜRASYON ---
DATA_FOLDER = '' 
MOVEMENT_TYPE = ''
TIMESTEPS = 80
NUM_SENSORS = 10
FEATURES_PER_SENSOR = 4 # w, x, y, z
FEATURES = NUM_SENSORS * FEATURES_PER_SENSOR
USERS_TO_ANALYZE = ["User-A", "User-B", "User-C", "User-D", "User-E"]
EPOCHS = 60
BATCH_SIZE = 16

# --- Adım 2: Yardımcı Fonksiyonlar ---

def normalize_sequence(sequence_data):
    """(N, 40) veriyi ilk karesine göre normalize eder."""
    if not sequence_data.any(): return sequence_data
    first_frame = sequence_data[0]
    inverse_references = []
    for i in range(NUM_SENSORS):
        offset = i * FEATURES_PER_SENSOR
        w, x, y, z = first_frame[offset:offset+4]
        q_ref = Quaternion(w, x, y, z) if (w or x or y or z) else Quaternion(1, 0, 0, 0)
        inverse_references.append(q_ref.inverse)
        
    normalized_sequence = []
    for frame in sequence_data:
        normalized_frame_features = []
        for i in range(NUM_SENSORS):
            offset = i * FEATURES_PER_SENSOR
            w, x, y, z = frame[offset:offset+4]
            q_live = Quaternion(w, x, y, z) if (w or x or y or z) else Quaternion(1, 0, 0, 0)
            q_normalized = inverse_references[i] * q_live
            normalized_frame_features.extend([q_normalized.w, q_normalized.x, q_normalized.y, q_normalized.z])
        normalized_sequence.append(normalized_frame_features)
    return np.array(normalized_sequence)

# --- Adım 3: Model Mimarilerini Oluşturan Fonksiyonlar ---
INPUT_SHAPE = (TIMESTEPS, FEATURES)

def build_cnn_lstm_model(num_classes):
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=INPUT_SHAPE),
        Conv1D(filters=64, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        BatchNormalization(),
        Dropout(0.3),
        LSTM(50, return_sequences=False),
        BatchNormalization(),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dense(num_classes, activation='softmax')
    ], name="CNN_LSTM")
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def build_cnn_gru_model(num_classes):
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=INPUT_SHAPE),
        Conv1D(filters=64, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        BatchNormalization(),
        Dropout(0.3),
        GRU(50, return_sequences=False),
        BatchNormalization(),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dense(num_classes, activation='softmax')
    ], name="CNN_GRU")
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def build_cnn_ann_model(num_classes):
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=INPUT_SHAPE),
        MaxPooling1D(pool_size=2),
        BatchNormalization(),
        Dropout(0.3),
        
        Conv1D(filters=128, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        BatchNormalization(),
        Dropout(0.3),
        
        Flatten(), # <-- GlobalAveragePooling yerine Flatten
        
        Dense(100, activation='relu'),
        Dropout(0.4),
        Dense(num_classes, activation='softmax')
    ], name="CNN_ANN_Hybrid")
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model
def build_pure_cnn_model(num_classes):
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=INPUT_SHAPE),
        BatchNormalization(),
        Conv1D(filters=64, kernel_size=3, activation='relu'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),
        Conv1D(filters=128, kernel_size=3, activation='relu'),
        BatchNormalization(),
        Conv1D(filters=128, kernel_size=3, activation='relu'),
        BatchNormalization(),
        GlobalAveragePooling1D(),
        Dropout(0.4),
        Dense(64, activation='relu'),
        Dense(num_classes, activation='softmax')
    ], name="Pure_1D_CNN")
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def build_transformer_model(num_classes):
    inputs = Input(shape=INPUT_SHAPE)
    x = Conv1D(filters=64, kernel_size=3, activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    
    attn_output = MultiHeadAttention(num_heads=4, key_dim=64)(x, x)
    attn_output = Dropout(0.2)(attn_output)
    x = Add()([x, attn_output])
    x = LayerNormalization(epsilon=1e-6)(x)
    
    ff_output = Dense(128, activation='relu')(x)
    ff_output = Dropout(0.2)(ff_output)
    ff_output = Dense(64, activation='relu')(ff_output)
    x = Add()([x, ff_output])
    x = LayerNormalization(epsilon=1e-6)(x)
    
    x = GlobalAveragePooling1D()(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs, name="Transformer")
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# --- Adım 4: Veriyi Yükleme, İşleme ve Ayırma ---
print("Adım 4: Veri Yükleniyor ve İşleniyor...")

all_data = []
all_labels = []
movement_path = os.path.join(DATA_FOLDER, MOVEMENT_TYPE)

for user_folder in USERS_TO_ANALYZE:
    user_path = os.path.join(movement_path, user_folder)
    if not os.path.isdir(user_path): continue
    for label_folder in os.listdir(user_path):
        label_path = os.path.join(user_path, label_folder)
        if os.path.isdir(label_path):
            for pkl_file in os.listdir(label_path):
                if pkl_file.endswith('.pkl'):
                    file_path = os.path.join(label_path, pkl_file)
                    try:
                        with open(file_path, 'rb') as f: obj = pickle.load(f)
                        recording_data = np.array(obj['data'])
                        if recording_data.shape[0] < 1 or recording_data.shape[1] != FEATURES: continue
                        
                        recording_data_normalized = normalize_sequence(recording_data)
                        recording_data_padded = pad_sequences(
                            [recording_data_normalized], maxlen=TIMESTEPS, dtype='float32', 
                            padding='post', truncating='post'
                        )[0]
                        
                        all_data.append(recording_data_padded)
                        all_labels.append(label_folder)
                    except Exception as e:
                        print(f"Hata: {file_path} dosyası okunurken hata oluştu: {e}")

print(f"Toplam {len(all_data)} adet orijinal .pkl kaydı yüklendi.")

# Etiketleri Kodla
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(all_labels)
num_classes = len(label_encoder.classes_)
y_one_hot = to_categorical(y_encoded, num_classes=num_classes)
X = np.array(all_data)

print(f"Bulunan sınıflar: {label_encoder.classes_}")

# Veriyi Rastgele Ayır (LOUOCV OLMADAN)
X_train, X_test, y_train, y_test = train_test_split(
    X, y_one_hot, 
    test_size=0.2, # 300 kaydın %20'si = 60 test kaydı
    #random_state=42, 
    stratify=y_one_hot # Sınıf dağılımını koru
)

# Ölçekle
scaler = StandardScaler()
X_train_reshaped = X_train.reshape(-1, FEATURES)
scaler.fit(X_train_reshaped)

X_train_scaled = scaler.transform(X_train_reshaped).reshape(X_train.shape)
X_test_scaled = scaler.transform(X_test.reshape(-1, FEATURES)).reshape(X_test.shape)

print(f"Eğitim verisi: {X_train_scaled.shape}, Test verisi: {X_test_scaled.shape}")
print("Veri Ayırma ve Ölçekleme Tamamlandı.\n")

# --- Adım 5: Modelleri Karşılaştırmalı Olarak Eğitme ve Değerlendirme ---
print("Adım 5: Karşılaştırmalı Model Testi Başlatılıyor...")

models_to_test = {
    "CNN-LSTM": build_cnn_lstm_model,
    "Pure 1D-CNN": build_pure_cnn_model,
    "Transformer": build_transformer_model,
    "ANN": build_cnn_ann_model,
    
    
}

results = {}
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

for model_name, build_fn in models_to_test.items():
    print(f"\n--- Model Eğitiliyor: {model_name} ---")
    model = build_fn(num_classes)
    
    model.fit(
        X_train_scaled, y_train,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_data=(X_test_scaled, y_test),
        callbacks=[early_stopping],
        verbose=0 # Logları gizle
    )
    
    # Değerlendir
    loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)
    print(f"Test Doğruluğu: {accuracy * 100:.2f}%")
    
    y_pred_probs = model.predict(X_test_scaled)
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_test_labels = np.argmax(y_test, axis=1)
    
    print("Classification Report:")
    print(classification_report(y_test_labels, y_pred, target_names=label_encoder.classes_))
    
    results[model_name] = accuracy

# --- Adım 6: Nihai Sonuçları Göster ---
print("\n\n==============================================")
print("     RASTGELE BÖLME (Mixed-User) TEST SONUÇLARI     ")
print("==============================================")
print(f"Test Edilen Hareket: {MOVEMENT_TYPE}")
print(f"Veri Seti: {len(USERS_TO_ANALYZE)} Kullanıcı, Toplam {len(all_labels)} Orijinal Kayıt")
print("Veri Artırma: KULLANILMADI")
print("----------------------------------------------")

for model_name, avg_accuracy in results.items():
    print(f"Model: {model_name.ljust(12)} | Kişiselleştirilmiş Doğruluk: {avg_accuracy * 100:.2f}%")

print("==============================================")

Adım 4: Veri Yükleniyor ve İşleniyor...
Toplam 131 adet orijinal .pkl kaydı yüklendi.
Bulunan sınıflar: ['RoundedBack' 'SupportHand' 'True']
Eğitim verisi: (104, 80, 40), Test verisi: (27, 80, 40)
Veri Ayırma ve Ölçekleme Tamamlandı.

Adım 5: Karşılaştırmalı Model Testi Başlatılıyor...

--- Model Eğitiliyor: CNN-LSTM ---
Test Doğruluğu: 100.00%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step
Classification Report:
              precision    recall  f1-score   support

 RoundedBack       1.00      1.00      1.00         9
 SupportHand       1.00      1.00      1.00         9
        True       1.00      1.00      1.00         9

    accuracy                           1.00        27
   macro avg       1.00      1.00      1.00        27
weighted avg       1.00      1.00      1.00        27


--- Model Eğitiliyor: Pure 1D-CNN ---
Test Doğruluğu: 88.89%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
Classification Report:
              precision    recall  f1-score   support

 RoundedBack       0.75      1.